# 📊 Tipos de Datos en Redis - Práctica con Python

*Autor: @mCárdenas 2025*

En este notebook exploraremos todos los tipos de datos de Redis con ejemplos prácticos.

## 🔌 Conexión a Redis

In [ ]:
import redis
import json
from datetime import datetime

# Conectar a Redis
r = redis.Redis(host='localhost', port=6379, decode_responses=True)

# Verificar conexión
print(f"✅ Conexión exitosa: {r.ping()}")
print(f"📊 Versión: {r.info('server')['redis_version']}")

## 📝 1. STRINGS

El tipo más básico. Puede almacenar texto, números o datos binarios.

In [ ]:
# Limpiar datos anteriores
r.flushdb()

# SET y GET básico
r.set('saludo', 'Hola, Redis!')
print(f"GET saludo: {r.get('saludo')}")

# SET con expiración (TTL)
r.setex('sesion:abc123', 60, 'datos_de_sesion')  # Expira en 60 segundos
print(f"TTL sesion:abc123: {r.ttl('sesion:abc123')} segundos")

# Contadores con INCR/DECR
r.set('visitas', 0)
r.incr('visitas')  # 1
r.incr('visitas')  # 2
r.incrby('visitas', 10)  # 12
print(f"Visitas: {r.get('visitas')}")

# MSET y MGET (múltiples operaciones)
r.mset({'color:1': 'rojo', 'color:2': 'azul', 'color:3': 'verde'})
print(f"Colores: {r.mget('color:1', 'color:2', 'color:3')}")

# SETNX - Solo si no existe
resultado = r.setnx('clave_unica', 'primer_valor')
print(f"Primera vez: {resultado}")
resultado = r.setnx('clave_unica', 'segundo_valor')  # No sobrescribe
print(f"Segunda vez: {resultado}")
print(f"Valor: {r.get('clave_unica')}")

## 📋 2. LISTS

Listas ordenadas, perfectas para colas y pilas.

In [ ]:
# LPUSH y RPUSH - Insertar elementos
r.delete('tareas')
r.rpush('tareas', 'tarea1', 'tarea2', 'tarea3')  # Añadir al final
r.lpush('tareas', 'tarea0')  # Añadir al inicio

# LRANGE - Ver la lista
print(f"Todas las tareas: {r.lrange('tareas', 0, -1)}")

# LPOP y RPOP - Extraer elementos
primera = r.lpop('tareas')
print(f"Primera extraída: {primera}")

ultima = r.rpop('tareas')
print(f"Última extraída: {ultima}")

# LLEN - Longitud
print(f"Tareas restantes: {r.llen('tareas')}")

# LINDEX - Acceso por índice
print(f"Elemento en índice 0: {r.lindex('tareas', 0)}")

# Ejemplo: Cola de trabajos (FIFO)
r.delete('cola_trabajos')
for i in range(5):
    r.rpush('cola_trabajos', f'trabajo_{i}')
    
print("\n📥 Cola de trabajos:")
while r.llen('cola_trabajos') > 0:
    trabajo = r.lpop('cola_trabajos')
    print(f"  Procesando: {trabajo}")

## 🔵 3. SETS

Conjuntos de elementos únicos, sin orden.

In [ ]:
# SADD - Añadir elementos
r.delete('frutas')
r.sadd('frutas', 'manzana', 'naranja', 'plátano', 'manzana')  # Duplicado ignorado

# SMEMBERS - Ver todos
print(f"Frutas: {r.smembers('frutas')}")

# SISMEMBER - ¿Pertenece?
print(f"¿Hay manzana? {r.sismember('frutas', 'manzana')}")
print(f"¿Hay uva? {r.sismember('frutas', 'uva')}")

# SCARD - Cardinalidad
print(f"Total frutas: {r.scard('frutas')}")

# Operaciones de conjuntos
r.delete('set_a', 'set_b')
r.sadd('set_a', 1, 2, 3, 4)
r.sadd('set_b', 3, 4, 5, 6)

print(f"\nSet A: {r.smembers('set_a')}")
print(f"Set B: {r.smembers('set_b')}")
print(f"Intersección (A ∩ B): {r.sinter('set_a', 'set_b')}")
print(f"Unión (A ∪ B): {r.sunion('set_a', 'set_b')}")
print(f"Diferencia (A - B): {r.sdiff('set_a', 'set_b')}")

## 🏆 4. SORTED SETS (ZSETS)

Sets ordenados por un score numérico. Ideal para rankings.

In [ ]:
# ZADD - Añadir con score
r.delete('leaderboard')
r.zadd('leaderboard', {
    'player1': 1500,
    'player2': 2000,
    'player3': 1800,
    'player4': 2200,
    'player5': 1600
})

# ZRANGE - Por posición (ascendente)
print("Top 3 (menor a mayor):")
print(r.zrange('leaderboard', 0, 2, withscores=True))

# ZREVRANGE - Por posición (descendente)
print("🏆 Top 3 (mayor a menor):")
for i, (player, score) in enumerate(r.zrevrange('leaderboard', 0, 2, withscores=True), 1):
    medalla = ['🥇', '🥈', '🥉'][i-1]
    print(f"  {medalla} {player}: {int(score)} pts")

# ZSCORE - Obtener score
print(f"Score de player3: {r.zscore('leaderboard', 'player3')}")

# ZRANK / ZREVRANK - Posición
print(f"Posición de player3 (0-indexed, desc): {r.zrevrank('leaderboard', 'player3')}")

# ZINCRBY - Actualizar score
r.zincrby('leaderboard', 500, 'player3')
print(f"Nuevo score player3: {r.zscore('leaderboard', 'player3')}")

# Nuevo ranking
print("🏆 Nuevo ranking:")
for i, (player, score) in enumerate(r.zrevrange('leaderboard', 0, 4, withscores=True), 1):
    print(f"  #{i} {player}: {int(score)} pts")

## 🗂️ 5. HASHES

Mapas campo-valor, ideales para objetos.

In [ ]:
# HSET - Establecer campos
r.delete('usuario:1')
r.hset('usuario:1', mapping={
    'nombre': 'María García',
    'email': 'maria@email.com',
    'edad': 28,
    'ciudad': 'Madrid',
    'visitas': 0
})

# HGET - Obtener un campo
print(f"Nombre: {r.hget('usuario:1', 'nombre')}")

# HGETALL - Obtener todo
print(f"Perfil completo:")
for campo, valor in r.hgetall('usuario:1').items():
    print(f"  {campo}: {valor}")

# HINCRBY - Incrementar campo numérico
r.hincrby('usuario:1', 'visitas', 1)
r.hincrby('usuario:1', 'edad', 1)  # Cumpleaños
print(f"Visitas: {r.hget('usuario:1', 'visitas')}")
print(f"Nueva edad: {r.hget('usuario:1', 'edad')}")

# HEXISTS - ¿Existe campo?
print(f"¿Tiene teléfono? {r.hexists('usuario:1', 'telefono')}")

# HDEL - Eliminar campo
r.hset('usuario:1', 'temporal', 'dato')
r.hdel('usuario:1', 'temporal')

## 🌊 6. STREAMS

Logs append-only para event sourcing.

In [ ]:
# XADD - Añadir eventos
r.delete('eventos')

# Simular eventos de usuario
eventos_ejemplo = [
    {'tipo': 'login', 'usuario': 'user123', 'ip': '192.168.1.1'},
    {'tipo': 'click', 'usuario': 'user123', 'pagina': '/productos'},
    {'tipo': 'compra', 'usuario': 'user123', 'producto': 'ABC', 'precio': '99.99'},
    {'tipo': 'logout', 'usuario': 'user123'}
]

for evento in eventos_ejemplo:
    id_evento = r.xadd('eventos', evento)
    print(f"Evento añadido: {id_evento}")

# XLEN - Longitud del stream
print(f"Total eventos: {r.xlen('eventos')}")

# XRANGE - Leer eventos
print("📜 Historial de eventos:")
for id_evento, campos in r.xrange('eventos', '-', '+'):
    print(f"  [{id_evento}] {campos}")

# XREAD - Leer desde un punto
print("📖 Últimos 2 eventos:")
eventos = r.xrevrange('eventos', '+', '-', count=2)
for id_evento, campos in eventos:
    print(f"  {campos['tipo']}: {campos}")

## 🔢 7. BITMAPS

Operaciones a nivel de bits. Muy eficiente en memoria.

In [ ]:
# Ejemplo: Registro de asistencia diaria
# Cada bit representa un usuario (0=ausente, 1=presente)
r.delete('asistencia:2024:01:15')

# Usuarios 1, 5, 10, 100 asistieron
usuarios_presentes = [1, 5, 10, 100, 150, 200]
for user_id in usuarios_presentes:
    r.setbit('asistencia:2024:01:15', user_id, 1)

# ¿Asistió usuario 5?
print(f"¿Usuario 5 asistió? {r.getbit('asistencia:2024:01:15', 5)}")

# ¿Asistió usuario 7?
print(f"¿Usuario 7 asistió? {r.getbit('asistencia:2024:01:15', 7)}")

# ¿Cuántos asistieron?
print(f"Total asistentes: {r.bitcount('asistencia:2024:01:15')}")

# Simular otro día
r.delete('asistencia:2024:01:16')
for user_id in [1, 10, 15, 100]:  # Menos gente
    r.setbit('asistencia:2024:01:16', user_id, 1)

# Usuarios que asistieron AMBOS días (AND)
r.bitop('AND', 'asistencia:ambos_dias', 'asistencia:2024:01:15', 'asistencia:2024:01:16')
print(f"Asistieron ambos días: {r.bitcount('asistencia:ambos_dias')}")

## 🎲 8. HYPERLOGLOG

Conteo probabilístico de elementos únicos. Máximo 12KB de memoria.

In [ ]:
import random
import string

# Simular visitantes únicos
r.delete('visitantes:hoy')

# Generar 10,000 visitas de ~5,000 usuarios únicos
usuarios_unicos = [f"user_{i}" for i in range(5000)]

for _ in range(10000):
    usuario = random.choice(usuarios_unicos)
    r.pfadd('visitantes:hoy', usuario)

# Contar únicos (aproximado)
conteo = r.pfcount('visitantes:hoy')
print(f"Visitantes únicos (aproximado): {conteo}")
print(f"Visitantes reales: 5000")
print(f"Error: {abs(conteo - 5000) / 5000 * 100:.2f}%")

# HyperLogLog usa solo ~12KB sin importar cuántos elementos
print(f"💡 Memoria usada: ~12KB (constante)")
print(f"   Si usáramos SET: ~{5000 * 10 / 1024:.1f}KB mínimo")

## 📍 9. GEOSPATIAL

Almacenamiento y consultas geográficas.

In [ ]:
# Añadir ubicaciones (longitud, latitud)
r.delete('restaurantes')

restaurantes = {
    'La Barraca': (-3.7037, 40.4168),      # Centro Madrid
    'El Corte': (-3.6897, 40.4200),        # Cerca
    'Casa Lucio': (-3.7074, 40.4138),      # Cerca
    'Sobrino Botin': (-3.7075, 40.4141),   # Muy cerca
    'DiverXO': (-3.6765, 40.4570),         # Norte (lejos)
}

for nombre, (lng, lat) in restaurantes.items():
    r.geoadd('restaurantes', (lng, lat, nombre))

# Obtener coordenadas
pos = r.geopos('restaurantes', 'La Barraca')
print(f"Coordenadas La Barraca: {pos}")

# Distancia entre dos restaurantes
dist = r.geodist('restaurantes', 'La Barraca', 'DiverXO', 'km')
print(f"Distancia La Barraca → DiverXO: {dist:.2f} km")

# Buscar restaurantes en radio de 1km desde La Barraca
print("🍽️ Restaurantes a menos de 1km de La Barraca:")
cercanos = r.geosearch('restaurantes', member='La Barraca', radius=1, unit='km', withdist=True)
for nombre, distancia in cercanos:
    print(f"  📍 {nombre}: {distancia:.2f} km")

# Buscar por coordenadas específicas
print("🍽️ Restaurantes a 500m de Puerta del Sol:")
cercanos = r.geosearch('restaurantes', longitude=-3.7037, latitude=40.4168, radius=500, unit='m', withdist=True)
for nombre, distancia in cercanos:
    print(f"  📍 {nombre}: {distancia:.0f} m")

## 📄 10. JSON (RedisStack)

Documentos JSON nativos con acceso parcial.

In [ ]:
# JSON requiere RedisStack
try:
    # Crear documento JSON complejo
    producto = {
        "nombre": "Laptop Gaming Pro",
        "precio": 1299.99,
        "stock": 50,
        "specs": {
            "cpu": "Intel i9",
            "ram": "32GB",
            "gpu": "RTX 4080"
        },
        "tags": ["gaming", "laptop", "premium"],
        "activo": True
    }
    
    # JSON.SET
    r.json().set('producto:json:1', '$', producto)
    print("✅ Producto creado")
    
    # JSON.GET - Todo el documento
    doc = r.json().get('producto:json:1')
    print(f"📦 Documento completo:")
    print(json.dumps(doc, indent=2, ensure_ascii=False))
    
    # JSON.GET - Acceso parcial
    precio = r.json().get('producto:json:1', '$.precio')
    print(f"💰 Precio: {precio}")
    
    cpu = r.json().get('producto:json:1', '$.specs.cpu')
    print(f"🖥️ CPU: {cpu}")
    
    # JSON.NUMINCRBY - Decrementar stock
    r.json().numincrby('producto:json:1', '$.stock', -5)
    stock = r.json().get('producto:json:1', '$.stock')
    print(f"📦 Nuevo stock: {stock}")
    
    # JSON.ARRAPPEND - Añadir tag
    r.json().arrappend('producto:json:1', '$.tags', 'oferta')
    tags = r.json().get('producto:json:1', '$.tags')
    print(f"🏷️ Tags: {tags}")
    
except Exception as e:
    print(f"⚠️ JSON requiere RedisStack: {e}")

## 🔍 11. SEARCH (RedisStack)

Búsqueda full-text e índices secundarios.

In [ ]:
from redis.commands.search.field import TextField, NumericField, TagField
from redis.commands.search.index_definition import IndexDefinition, IndexType
from redis.commands.search.query import Query

try:
    # Eliminar índice si existe
    try:
        r.ft('idx:productos').dropindex(delete_documents=True)
    except:
        pass
    
    # Crear índice
    schema = (
        TextField('nombre', sortable=True),
        NumericField('precio', sortable=True),
        TagField('categoria')
    )
    
    r.ft('idx:productos').create_index(
        schema,
        definition=IndexDefinition(prefix=['producto:'], index_type=IndexType.HASH)
    )
    print("✅ Índice creado")
    
    # Crear productos
    productos = [
        {'nombre': 'Laptop Gaming RTX', 'precio': 1500, 'categoria': 'electronica'},
        {'nombre': 'Laptop Oficina', 'precio': 800, 'categoria': 'electronica'},
        {'nombre': 'Monitor 4K', 'precio': 400, 'categoria': 'electronica'},
        {'nombre': 'Silla Ergonómica', 'precio': 300, 'categoria': 'muebles'},
        {'nombre': 'Mesa Gaming RGB', 'precio': 250, 'categoria': 'muebles'},
    ]
    
    for i, prod in enumerate(productos, 1):
        r.hset(f'producto:{i}', mapping=prod)
    print(f"✅ {len(productos)} productos creados")
    
    # Buscar por texto
    print("🔍 Búsqueda: 'Laptop'")
    result = r.ft('idx:productos').search('Laptop')
    for doc in result.docs:
        print(f"  - {doc.nombre} (${doc.precio})")
    
    # Buscar con filtro numérico
    print("🔍 Búsqueda: precio < 500")
    result = r.ft('idx:productos').search('@precio:[0 500]')
    for doc in result.docs:
        print(f"  - {doc.nombre} (${doc.precio})")
    
    # Buscar por categoría
    print("🔍 Búsqueda: categoría = muebles")
    result = r.ft('idx:productos').search('@categoria:{muebles}')
    for doc in result.docs:
        print(f"  - {doc.nombre} (${doc.precio})")
        
except Exception as e:
    print(f"⚠️ Search requiere RedisStack: {e}")

## 🧹 Limpieza

In [ ]:
# Opcional: Limpiar todas las claves creadas
# r.flushdb()
# print("✅ Base de datos limpiada")

print(f"📊 Total claves en DB: {r.dbsize()}")

## 📚 Resumen

| Tipo | Uso Principal | Comando Clave |
|------|---------------|---------------|
| **String** | Valores simples, contadores | `SET`, `GET`, `INCR` |
| **List** | Colas, pilas, historial | `LPUSH`, `RPOP`, `LRANGE` |
| **Set** | Elementos únicos, tags | `SADD`, `SMEMBERS`, `SINTER` |
| **Sorted Set** | Rankings, ordenación | `ZADD`, `ZREVRANGE`, `ZINCRBY` |
| **Hash** | Objetos, perfiles | `HSET`, `HGET`, `HGETALL` |
| **Stream** | Event sourcing, logs | `XADD`, `XREAD`, `XRANGE` |
| **Bitmap** | Flags, presencia | `SETBIT`, `GETBIT`, `BITCOUNT` |
| **HyperLogLog** | Conteo aproximado | `PFADD`, `PFCOUNT` |
| **Geospatial** | Ubicaciones | `GEOADD`, `GEOSEARCH` |
| **JSON** | Documentos complejos | `JSON.SET`, `JSON.GET` |
| **Search** | Búsqueda full-text | `FT.SEARCH`, `FT.CREATE` |